In [ ]:
import requests
start = 10000
for i in range(start, start+100000):
    idd = i
    url = f"http://localhost:8003/api/v1/tags/{idd}/"
    r = requests.delete(url) 

## Delete tags

In [ ]:
import requests    
def delete_tags(): 
    url = "http://localhost:8003/api/v1/tags/"
    r = requests.get(url)
    for t in r.json():
        idd = t['id']
        urld = f"http://localhost:8003/api/v1/tags/{idd}/"
        r = requests.delete(urld)

## Analyze steps

In [ ]:
import json
import requests
import glob
files = glob.glob("../data/path/steps/*")
files_txt = glob.glob("../data/path/txt/*")
files_html = glob.glob("../data/path/html/*")
ip = ""

do_geo = False
do_words = False
do_tags = False

do_geo = True
do_words = True
do_tags = True

delete_tags()

def get_gext(html):
    html = html.encode('utf-8')
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, "html.parser", from_encoding="utf-8")           
    # headings = [h.get_text() for h in soup.find_all(['h1', 'h2', 'h3'])]
    # head = " ".join(headings)
    paragraphs = [p.get_text() for p in soup.find_all('p')]
    # text_out = head + " ".join(paragraphs)
    text_out = " ".join(paragraphs)    
    return text_out


import spacy
nlp = spacy.load("en_core_web_lg")


with open(f'../data/path/collect.tsv', 'w') as out_file:
                    
    for file in files:
        contents = open(file).readlines()
        step = int(contents[0].strip())
        
        
        # Analyze text
        text = open(files_txt[step-1]).read().strip().replace("\n","")        
        html = open(files_html[step-1]).read()
        text_new = get_gext(html).strip().replace("\n","") 
        text = text_new
        
        
        doc = nlp(text)
        
        # Find named entities, phrases and concepts
        # for entity in doc.ents:
            # print(entity.text, entity.label_)
        # print(text)
        # print(text_new)
        # print("------------")
        
        code = contents[3].strip()
        if code == "200":
            ip = contents[4].strip()        
        latitude = "0"
        longitude = "0"
        city = ""
        
        if do_geo:                
            url = f"http://localhost:8004/api/v1/ip/{ip}/"
            r = requests.get(url)
            latitude = str(r.json()['latitude'])
            longitude = str(r.json()['longitude'])
            city = r.json()['city']
        
        words = []
        hrases = []
        sim = []
                    
        if text != "":            
            if do_words:
                url_semantic = f"http://localhost:8005/api/v1/semantic/tags/"
                headers = {'content-type': 'application/json'}
                rr = requests.post(url_semantic, data = json.dumps({"text": text}), headers=headers)                
                data = rr.json()
                if "words" in data.keys(): 
                    words = rr.json()['words']
                if "hrases" in data.keys(): 
                    hrases = rr.json()['hrases']
                if "sim" in data.keys(): 
                    sim = rr.json()['sim']
                                    

            if do_tags:              
                for hras in sim:       
                    url = "http://localhost:8003/api/v1/tags/"
                    headers = {'content-type': 'application/json'}
                    data = {'name': hras, "count": 0}
                    response = requests.post(url, data=json.dumps(data), headers=headers)

        out = [contents[0].strip(), contents[1].strip(), contents[2].strip(), contents[3].strip(), contents[4].strip(), latitude, longitude, city, json.dumps(words), json.dumps(hrases), json.dumps(sim)] #text
        out_file.write('\t'.join(out)+"\n")
        # print(text)
        print(out)        
        
        # if step > 100:
            # break
    